In [34]:
import psycopg2
import pandas as pd
from config import config


In [35]:
params = config()
conn = psycopg2.connect(**params)
cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table



In [44]:
df = create_pandas_table("""SELECT * FROM iv_dataset WHERE "Ticker" = 'GME'""" )
df = df.set_index(['Date'])
df = df.filter(like='Call', axis=1)

In [45]:
df

,IvCall10,IvCall20,IvCall30,IvCall60,IvCall90,IvCall120,IvCall150,IvCall180,IvCall270,IvCall360,IvCall720,IvCall1080
Date,,,,,,,,,,,,
2002-10-04,0.6980,0.6928,0.6824,0.6646,0.6570,0.6483,0.6382,0.6282,0.6225,0.6225,0.6225,0.6225
2002-10-07,0.6783,0.6796,0.6813,0.6712,0.6535,0.6415,0.6339,0.6263,0.6227,0.6227,0.6227,0.6227
2002-10-08,0.6058,0.6254,0.6471,0.6652,0.6631,0.6567,0.6476,0.6384,0.6344,0.6344,0.6344,0.6344
2002-10-09,0.6373,0.6382,0.6391,0.6418,0.6446,0.6421,0.6366,0.6311,0.6289,0.6289,0.6289,0.6289
2002-10-10,0.6897,0.6933,0.6968,0.6830,0.6617,0.6549,0.6554,0.6558,0.6560,0.6560,0.6560,0.6560
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-12,1.9488,1.8740,1.8150,1.6977,1.5938,1.5149,1.4398,1.3648,1.2353,1.2043,1.1264,1.1264
2021-04-13,1.7074,1.7181,1.6969,1.6318,1.5385,1.4586,1.3801,1.3016,1.2016,1.1658,1.0673,1.0673
2021-04-14,1.8794,1.8008,1.7689,1.6514,1.5541,1.4802,1.4079,1.3356,1.2323,1.1830,1.0378,1.0378


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import math

###############################################################################
# Define required functions to describe the optimization problem
###############################################################################

# Double integral transformed
def sum_j_aK(a, x, z, h):
    j = len(a)
    loc = z
    scale = h
    x_normalized = (np.ones(j)*x - loc) / scale
    K_j = (x_normalized*norm.cdf(x_normalized) + np.exp(-0.5*x_normalized**2)/((2*np.pi)**0.5)) * scale
    return np.sum(a*K_j)

# Minimization problem
def sum_squared_pricing_diff(a, P, X, z, h):
    total = 0
    for i in range(0, len(P)):
        p = P[i]
        x = X[i]
        total += abs(p - sum_j_aK(a, x, z, h))
    return total

###############################################################################
# Input required to solve the optimization problem
###############################################################################

# P is an array of vector put option prices
P = [0.249999283, 0.43750315, 0.572923413, 0.760408034, 0.94790493, 1.14584317,
     1.458335038, 1.77083305, 2.624999786, 3.812499791, 5.377596753, 8.06065865,
     10.74376984, 14.88873497, 19.88822895]

# X is the vector of the corresponding strikes of the put options
X = [560, 570, 575, 580, 585, 590, 595, 600, 605, 610, 615, 620, 625, 630, 635]

# h and j can be chosen arbitrarily
h = 4 # the higher h the smoother the estimated risk-neutral density
j = 50 # the higher the slower the optimization process

###############################################################################
# Solving the optimization problem
###############################################################################

# z is the equally-spaced grid
z = np.linspace((int(math.floor(min(X) / 100.0)) * 100), (int(math.ceil(max(X) / 100.0)) * 100), num=j)

# initial guess of a
a_0 = np.ones(j) / j

# The a vector has to sum up to 1
constraints = ({'type': 'eq', 'fun': lambda a: 1 - np.sum(a)},)

# Each a has to be larger or equal than 0 
bounds = (((0,None),)*j)

sol = minimize(sum_squared_pricing_diff, a_0, args=(P, X, z, h), method='SLSQP', constraints=constraints, bounds=bounds)
print(sol)

###############################################################################
# Visualize obtained risk-neutral density (rnd)
###############################################################################
n = 500
a_sol = sol.x
s = np.linspace(min(X)*0.8, max(X)*1.2, num=n)

rnd = pd.DataFrame(np.sum(a_sol * norm.pdf(np.tile(s, [len(a_sol),1]).transpose(), z, h), axis=1))
rnd.index = s

plt.figure()
plt.plot(rnd)
options
python
optimization
risk-neutral-measure
density
